In [15]:
!pip install --upgrade scikit-learn==1.3.2 scikeras==0.12.0 tensorflow==2.12.0

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier


In [17]:
data=pd.read_csv("/content/Alphabets_data.csv")

In [18]:
x=data.iloc[:,1:]
y=data.iloc[:,0]

In [19]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le=LabelEncoder()
y_encod=le.fit_transform(y)
len(np.unique(y_encod))

26

In [20]:
scaler=StandardScaler()
x_scaler=scaler.fit_transform(x)

In [21]:
x_train,x_test,y_train,y_test=train_test_split(x_scaler,y_encod,test_size=0.2,random_state=42)
x_train.shape,y_train.shape

((16000, 16), (16000,))

In [22]:
def create_model():
  model=Sequential(name='hyperparameter_tunning_dummy_model')
  model.add(Dense(64,input_dim=16,kernel_initializer='uniform',activation='relu'))
  model.add(Dense(32,kernel_initializer='uniform',activation='relu'))
  model.add(Dense(26,kernel_initializer='uniform',activation='softmax'))
  adam=Adam(learning_rate=0.01)
  model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
  return model


In [23]:
default_model = create_model()
history=default_model.fit(x_train, y_train,)

500/500 [==============================] - 2s 3ms/step - loss: 1.0903 - accuracy: 0.6536


In [24]:
y_pred=default_model.predict(x_test)

125/125 [==============================] - 0s 2ms/step


In [25]:
from sklearn.metrics import classification_report
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test,y_pred_classes))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92       149
           1       0.65      0.92      0.76       153
           2       0.98      0.70      0.82       137
           3       0.83      0.81      0.82       156
           4       0.67      0.87      0.76       141
           5       0.70      0.80      0.75       140
           6       0.77      0.82      0.80       160
           7       0.79      0.64      0.71       144
           8       0.87      0.78      0.82       146
           9       0.94      0.80      0.86       149
          10       0.76      0.68      0.72       130
          11       0.93      0.88      0.91       155
          12       0.82      0.95      0.88       168
          13       1.00      0.64      0.78       151
          14       0.73      0.82      0.78       145
          15       0.98      0.78      0.87       173
          16       0.86      0.81      0.84       166
          17       0.69    

In [26]:
# accuracy 79%

In [14]:
from sklearn.model_selection import GridSearchCV
model=KerasClassifier(model=create_model,verbose=0)
params={'batch_size':[32,64],
        'epochs':[10,20]}
gridsearch=GridSearchCV(estimator=model,param_grid=params,cv=3)
grid_fit=gridsearch.fit(x_train,y_train)


In [27]:
print('Best : {}, using {}'.format(grid_fit.best_score_,grid_fit.best_params_))
means = grid_fit.cv_results_['mean_test_score']
stds = grid_fit.cv_results_['std_test_score']
params = grid_fit.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9121873547917891, using {'batch_size': 64, 'epochs': 20}
0.8838122452019581,0.010434334001023976 with: {'batch_size': 32, 'epochs': 10}
0.9008132726143154,0.009874401026488493 with: {'batch_size': 32, 'epochs': 20}
0.8963750265325215,0.0016353642301724374 with: {'batch_size': 64, 'epochs': 10}
0.9121873547917891,0.001644625927247022 with: {'batch_size': 64, 'epochs': 20}


In [28]:
def create_model1(learning_rate,dropout_rate):
  model=Sequential()
  model.add(Dense(64,input_dim=16,kernel_initializer='uniform',activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(32,kernel_initializer='uniform',activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(26,kernel_initializer='uniform',activation='softmax'))

  adam=Adam(learning_rate=learning_rate)
  model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
  return model

In [29]:
model=KerasClassifier(model=create_model1,verbose=0,batch_size=64,epochs=20,dropout_rate=[0.1,0.2,0.01],learning_rate=[0.001,0.01,0.1])
dropout_rate=[0.1,0.2,0.01]
learning_rate=[0.001,0.01,0.1]

params=dict(learning_rate=learning_rate,
            dropout_rate=dropout_rate)
gridsearch=GridSearchCV(estimator=model,param_grid=params,cv=3)
grid_fit=gridsearch.fit(x_train,y_train)


In [30]:
print('Best : {}, using {}'.format(grid_fit.best_score_,grid_fit.best_params_))
means = grid_fit.cv_results_['mean_test_score']
stds = grid_fit.cv_results_['std_test_score']
params = grid_fit.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9146878625920869, using {'dropout_rate': 0.01, 'learning_rate': 0.01}
0.8335003502398314,0.003989042508450065 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.8970634014832152,0.011556938012072209 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.11100121954097573,0.017284687230348744 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.8063756937564502,0.007850451325482895 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.8680627372356629,0.0077518647660239645 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.03856246124119165,0.0005348355902260874 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}
0.8595621825164224,0.0037709931606170304 with: {'dropout_rate': 0.01, 'learning_rate': 0.001}
0.9146878625920869,0.004985081133369047 with: {'dropout_rate': 0.01, 'learning_rate': 0.01}
0.3373082607126762,0.04796303847160969 with: {'dropout_rate': 0.01, 'learning_rate': 0.1}


In [31]:
def create_model2(init,activation_function):
    model = Sequential()
    model.add(Dense(64, input_dim=16, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(0.01))
    model.add(Dense(32, kernel_initializer=init, activation=activation_function))
    model.add(Dropout(0.01))
    model.add(Dense(26, kernel_initializer=init, activation='softmax'))
    adam = Adam(learning_rate=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [32]:
model2 = KerasClassifier(model=create_model2, verbose=0, batch_size=64, epochs=20, init='uniform',
    activation_function='relu')

# Define parameter grid
params = {
    'init': ['uniform', 'normal', 'zeros'],
    'activation_function': ['relu', 'tanh']
}

# Run Grid Search
gridsearch = GridSearchCV(estimator=model2, param_grid=params, cv=3)
grid_fit = gridsearch.fit(x_train, y_train)

In [33]:
print('best:{},using:{}'.format(grid_fit.best_score_,grid_fit.best_params_))
means = grid_fit.cv_results_['mean_test_score']
stds = grid_fit.cv_results_['std_test_score']
params = grid_fit.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))


best:0.9166246399961303,using:{'activation_function': 'relu', 'init': 'normal'}
0.9078131476768104,0.00794168017996722 with: {'activation_function': 'relu', 'init': 'uniform'}
0.9166246399961303,0.006371197447898104 with: {'activation_function': 'relu', 'init': 'normal'}
0.03818755498240387,0.0007732503836559758 with: {'activation_function': 'relu', 'init': 'zeros'}
0.9072500266174773,0.005902140632309513 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.9093750539761191,0.0007641215957687573 with: {'activation_function': 'tanh', 'init': 'normal'}
0.03993745343992136,0.0005485654638258446 with: {'activation_function': 'tanh', 'init': 'zeros'}


In [34]:
def create_model3(nuron1,nuron2):
  model=Sequential()
  model.add(Dense(nuron1,input_dim=16,kernel_initializer='normal',activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(nuron2,input_dim=nuron1,kernel_initializer='normal',activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(26,kernel_initializer='normal',activation='softmax'))

  adam=Adam(learning_rate=0.01)
  model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
  return model

In [35]:
model3=KerasClassifier(model=create_model3,batch_size=64,epochs=20,verbose=0,nuron1=[32,64],nuron2=[16,32])
nuron1=[32,64]
nuron2=[16,32]
params=dict(nuron1=nuron1,nuron2=nuron2)
gridsearch=GridSearchCV(estimator=model3,param_grid=params,cv=3,verbose=0)
grid_fit=gridsearch.fit(x_train,y_train)

In [36]:
print('best:{},using:{}'.format(grid_fit.best_score_,grid_fit.best_params_))
means = grid_fit.cv_results_['mean_test_score']
stds = grid_fit.cv_results_['std_test_score']
params = grid_fit.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

best:0.9128751204055616,using:{'nuron1': 64, 'nuron2': 32}
0.8719996630835546,0.00518937490621161 with: {'nuron1': 32, 'nuron2': 16}
0.8948752140090871,0.00433674570175219 with: {'nuron1': 32, 'nuron2': 32}
0.8969998078010734,0.004738691788119254 with: {'nuron1': 64, 'nuron2': 16}
0.9128751204055616,0.0050114315852511785 with: {'nuron1': 64, 'nuron2': 32}


In [37]:
def create_model4():
  model=Sequential()
  model.add(Dense(64,input_dim=16,kernel_initializer='normal',activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(32,kernel_initializer='normal',activation='relu'))
  model.add(Dropout(0.01))
  model.add(Dense(26,kernel_initializer='normal',activation='softmax'))

  adam=Adam(learning_rate=0.01)
  model.compile(loss='sparse_categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
  return model

In [38]:
model=KerasClassifier(model=create_model4,batch_size=64,epochs=20,verbose=0)
nural_model=model.fit(x_train,y_train)
y_pred=nural_model.predict(x_test)


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       149
           1       0.89      0.96      0.92       153
           2       0.96      0.91      0.94       137
           3       0.94      0.87      0.91       156
           4       0.89      0.95      0.92       141
           5       0.97      0.88      0.92       140
           6       0.95      0.91      0.93       160
           7       0.89      0.87      0.88       144
           8       0.91      0.95      0.93       146
           9       0.96      0.91      0.93       149
          10       0.91      0.87      0.89       130
          11       0.95      0.96      0.96       155
          12       0.98      0.98      0.98       168
          13       0.95      0.95      0.95       151
          14       0.92      0.97      0.94       145
          15       0.96      0.95      0.95       173
          16       0.97      0.93      0.95       166
          17       0.80    

In [40]:
# model have 93% accuracy

In [ ]:
✅ Key Differences in Metrics:
Metric	        Default Model	                    Tuned Model
Training         Accuracy	Moderate (79%)	        High (93%)
Validation       AccuracyLow/Fluctuating	        Improved/Stable
Generalization	  Poor (overfit/underfit)	         Better balance
Training           Time	Short	                     Longer (but worth it)
Loss Convergence	   May plateau early	           More gradual & stable

